In [ ]:
import pandas as pd
import re
from scipy.spatial.transform import Rotation as R
import matplotlib.pyplot as plt

def parse_colmap_cameras(cameras_txt_path):
    cameras = pd.read_csv(
        cameras_txt_path, 
        comment='#', 
        delim_whitespace=True, 
        header=None,
        names=['CAMERA_ID', 'MODEL', 'WIDTH', 'HEIGHT', 'PARAM1', 'PARAM2', 'PARAM3', 'PARAM4']
    )
    return cameras

def parse_colmap_images(images_txt_path):
    images = pd.read_csv(
        images_txt_path, 
        comment='#', 
        delim_whitespace=True, 
        header=None,
        names=['IMAGE_ID', 'QW', 'QX', 'QY', 'QZ', 'TX', 'TY', 'TZ', 'CAMERA_ID', 'NAME']
    )
    return images

def parse_leica_file(leica_txt_path):
    with open(leica_txt_path, 'r') as file:
        lines = file.readlines()
    
    timestamps = []
    x_coords = []
    y_coords = []
    z_coords = []
    
    pattern = re.compile(r'%R1P.*,([-.\d]+),([-.\d]+),([-.\d]+),.*')

    for line in lines:
        if '%R1P' in line:
            match = pattern.search(line)
            if match:
                x, y, z = match.group(1), match.group(2), match.group(3)
                timestamps.append(line.split(',')[1])  # Adjust based on actual position
                x_coords.append(float(x))
                y_coords.append(float(y))
                z_coords.append(float(z))
    
    leica_df = pd.DataFrame({
        'Timestamp': pd.to_datetime(timestamps),
        'X': x_coords,
        'Y': y_coords,
        'Z': z_coords
    })
    
    # Convert units if necessary
    leica_df[['X', 'Y', 'Z']] /= 1000  # Example conversion from mm to meters
    return leica_df

def compare_positions(colmap_images, leica_data):
    # Assuming synchronization is handled
    comparison_df = pd.concat([
        colmap_images[['TX', 'TY', 'TZ']].reset_index(drop=True),
        leica_data[['X', 'Y', 'Z']].reset_index(drop=True)
    ], axis=1)
    
    comparison_df.columns = ['COLMAP_TX', 'COLMAP_TY', 'COLMAP_TZ', 'Leica_X', 'Leica_Y', 'Leica_Z']
    
    # Compute differences
    comparison_df['Diff_X'] = comparison_df['COLMAP_TX'] - comparison_df['Leica_X']
    comparison_df['Diff_Y'] = comparison_df['COLMAP_TY'] - comparison_df['Leica_Y']
    comparison_df['Diff_Z'] = comparison_df['COLMAP_TZ'] - comparison_df['Leica_Z']
    
    return comparison_df

def main():
    # Paths (update these paths accordingly)
    colmap_cameras_path = 'D:/Lectures/Robot/Homeworks/HW3\code/cameras.txt'
    colmap_images_path = 'D:/Lectures/Robot/Homeworks/HW3/code/images.txt'
    leica_txt_path = 'D:/Lectures/Robot/Homeworks/HW3/code/leica.txt'
    
    # Parse files
    colmap_cameras = parse_colmap_cameras(colmap_cameras_path)
    colmap_images = parse_colmap_images(colmap_images_path)
    leica_data = parse_leica_file(leica_txt_path)
    
    # Compare positions
    comparison_df = compare_positions(colmap_images, leica_data)
    
    # Print summary
    print(comparison_df[['Diff_X', 'Diff_Y', 'Diff_Z']].describe())
    
    # Plot differences
    plt.figure(figsize=(12, 6))
    plt.plot(comparison_df['Diff_X'], label='Delta X')
    plt.plot(comparison_df['Diff_Y'], label='Delta Y')
    plt.plot(comparison_df['Diff_Z'], label='Delta Z')
    plt.xlabel('Frame/Image Index')
    plt.ylabel('Difference (m)')
    plt.title('Position Differences Between COLMAP and Leica')
    plt.legend()
    plt.show()
    
    # 3D Scatter Plot
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')
    ax.scatter(comparison_df['COLMAP_TX'], comparison_df['COLMAP_TY'], comparison_df['COLMAP_TZ'], c='r', label='COLMAP')
    ax.scatter(comparison_df['Leica_X'], comparison_df['Leica_Y'], comparison_df['Leica_Z'], c='b', label='Leica')
    ax.set_xlabel('X (m)')
    ax.set_ylabel('Y (m)')
    ax.set_zlabel('Z (m)')
    ax.legend()
    plt.show()

if __name__ == "__main__":
    main()